<a href="https://colab.research.google.com/github/Shellyga/Adversarial-Domain-Adaptation-with-Keras/blob/master/new%20implementation%20adda.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Load images

In [1]:
SEED = 7
import os
import sys
import argparse
import random
import numpy as np
# import tensorflow.python.keras as tf
# from tensorflow.compat.v1 import set_random_seed
# import tensorflow.python.keras as tf
from sklearn.model_selection import train_test_split
from sklearn import metrics
from plotnine import *
import pandas as pd
                
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from math import ceil
from sklearn.utils import shuffle as skshuffle

from imutils import paths
from sklearn.preprocessing import LabelEncoder

# os.environ['PYTHONHASHSEED']=str(SEED)
# np.random.seed(SEED)
# set_random_seed(SEED)
# random.seed(SEED)

from PIL import Image
from keras.utils import to_categorical
from keras.layers import Input
from keras.optimizers import Adam
from keras.utils import multi_gpu_model
from sklearn.metrics import accuracy_score

NUM_CLASSES = 0


def pil_loader(path):
    # print(path)
    # Return the RGB variant of input image
    with open(path, 'rb') as f:
      with Image.open(f) as img:
        return img.convert('RGB')

def one_hot_encoding(labels):
    lb = LabelEncoder()
    fit_labels  = lb.fit_transform(labels)
    NUM_CLASSES = len(lb.classes_)
    labels = to_categorical(fit_labels)
    return labels

            
def data_loader(filepath, inp_dims):
    # Load images and corresponding labels from the text file, stack them in numpy arrays and return
    # if not os.path.isfile(filepath):
    #     print("File path {} does not exist. Exiting...".format(filepath))
        # sys.exit() 
    img = []
    label = []
    detector = dlib.get_frontal_face_detector()
    predictor = "drive/My Drive/final_proj_dataset/shape_predictor_5_face_landmarks.dat"
    # grab the image paths
    imagePaths = sorted(list(paths.list_images(filepath)))
    print(imagePaths)
    # loop over the input images
    for imagePath in imagePaths:
        # extract the class label from the image path and update the
        # labels list
        l = imagePath.split(os.path.sep)[-2]
        label.append(l)
        i = pil_loader(imagePath)
        i = i.resize((inp_dims[0], inp_dims[1]), Image.ANTIALIAS)
        frame_1 = align_and_crop(detector, np.array(i), predictor)[0]
        frame_1 = cv2.resize(frame_1,(inp_dims[0], inp_dims[1]))
        img.append(frame_1)
    img = np.array(img)
    label = np.array(label)
    return img, label

def get_dataset(path):
      print("[INFO] loading images...")
      # Load source and target data
      X,y = data_loader(path, [224,224,3])
      print("images loaded")
      X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42)
      # Encode labels into one-hot format
      print("[INFO] Encode labels into one-hot format")
      y_train = one_hot_encoding(y_train)
      y_test = one_hot_encoding(y_test)
      return (X_train, y_train), (X_test, y_test)

Using TensorFlow backend.


In [19]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


# Crop

In [0]:
import dlib

import os
import sys
import cv2
import glob


def align_and_crop(detector, im_to_align, predictor_path):
    '''
    simply aligns the photo (using the detector to identify the eyes) and crops the face.
    :param im_to_align: an address of an image
    '''

    # a shape predictor to find face landmarks so we can precisely localize the face
    sp = dlib.shape_predictor(predictor_path)

    # Ask the detector to find the bounding boxes of each face. The 1 in the
    # second argument indicates that we should upsample the image 1 time.
    try:
        dets = detector(im_to_align, 1)
    except RuntimeError:
        return -1

    num_faces = len(dets)
    if num_faces == 0:
        return im_to_align, False

    # Find the 5 face landmarks we need to do the alignment.
    faces = dlib.full_object_detections()
    for detection in dets:
        faces.append(sp(im_to_align, detection))

    # get a single chip (aligned and cropped)
    image = dlib.get_face_chip(im_to_align, faces[0])
    # cv2.imshow("f", image) show image for testing
    return image, True

# optimizer

In [0]:
import numpy as np
from keras.optimizers import Adam

def opt_classifier(param):
    return Adam(lr=param["lr_classifier"], beta_1=param["b1_classifier"], beta_2=param["b2_classifier"])

def opt_discriminator(param):
    return Adam(lr=param["lr_discriminator"], beta_1=param["b1_discriminator"], beta_2=param["b2_discriminator"])



# ADDA

In [0]:
from __future__ import print_function, division

from keras.datasets import mnist
from keras.layers import Input, Dense, Reshape, Flatten, Dropout
from keras.layers import BatchNormalization, Activation, ZeroPadding2D, MaxPooling2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model, clone_model
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from keras import regularizers
from keras.utils import np_utils
import keras.backend as K
import keras

import tensorflow.compat.v1 as tf
# from datasets import get_dataset

import matplotlib.pyplot as plt

import sys
import os
import numpy as np
import argparse
from keras_vggface.vggface import VGGFace

class ADDA():
    def __init__(self, lr):
        # Input shape
        self.img_rows = 224
        self.img_cols = 224
        self.channels = 3
        self.img_shape = (self.img_rows, self.img_cols, self.channels)
        
        self.src_flag = False
        self.disc_flag = False
        
        self.discriminator_decay_rate = 3 #iterations
        self.discriminator_decay_factor = 0.5
        self.src_optimizer = Adam(lr, 0.5)
        self.tgt_optimizer = Adam(lr, 0.5)
        
    def define_source_encoder(self, weights=None):
    
        #self.source_encoder = keras.applications.vgg16.VGG16(include_top=False, weights='imagenet', input_shape=self.img_shape, pooling=None, classes=10)
        
        # self.source_encoder = Sequential()
        inp = Input(shape=self.img_shape)
        # x = Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=self.img_shape, padding='same')(inp)
        # x = Conv2D(64, kernel_size=(3, 3), activation='relu', padding='same')(x)
        # x = MaxPooling2D(pool_size=(2, 2))(x)
        # x = Conv2D(256, kernel_size=(3, 3), activation='relu', padding='same')(x)
        # x = Conv2D(128, kernel_size=(3, 3), activation='relu', padding='same')(x)
        # x = Conv2D(64, kernel_size=(3, 3), activation='relu', padding='same')(x)
        # x = MaxPooling2D(pool_size=(2, 2))(x)
        # x = Conv2D(128, kernel_size=(3, 3), activation='relu', input_shape=self.img_shape, padding='same')(inp)
        # x = Conv2D(64, kernel_size=(3, 3), activation='relu', padding='same')(x)
        # x = Conv2D(32, kernel_size=(3, 3), activation='relu', padding='same')(x)
        # x = MaxPooling2D(pool_size=(2, 2))(x)
        # self.source_encoder = Model(inputs=(inp), outputs=(x))
        network = eval('VGGFace')
        base = network(weights = 'vggface', include_top = False)
        # base = network(weights = 'imagenet', include_top = False)
        self.source_encoder = Model(inp,base(inp))
        
        
        self.src_flag = True
        
        if weights is not None:
            self.source_encoder.load_weights(weights, by_name=True)
    
    def define_target_encoder(self, weights=None):
        
        if not self.src_flag:
            self.define_source_encoder()
        
        with tf.device('/cpu:0'):
            self.target_encoder = clone_model(self.source_encoder)
        
        if weights is not None:
            self.target_encoder.load_weights(weights, by_name=True)
        
    def get_source_classifier(self, model, weights=None):
        
        x = Flatten()(model.output)
        x = Dense(128, activation='relu')(x)
        x = Dense(10, activation='softmax')(x)
        
        source_classifier_model = Model(inputs=(model.input), outputs=(x))
        
        if weights is not None:
            source_classifier_model.load_weights(weights)
        
        return source_classifier_model
    
    def define_discriminator(self, shape):
        
        inp = Input(shape=shape)
        x = Flatten()(inp)
        x = Dense(128, activation=LeakyReLU(alpha=0.3), kernel_regularizer=regularizers.l2(0.01), name='discriminator1')(x)
        
        x = Dense(2, activation='sigmoid', name='discriminator2')(x)
        
        self.disc_flag = True
        self.discriminator_model = Model(inputs=(inp), outputs=(x), name='discriminator')
    
    def tensorboard_log(self, callback, names, logs, batch_no):
        
        for name, value in zip(names, logs):
            summary = tf.Summary()
            summary_value = summary.value.add()
            summary_value.simple_value = value
            summary_value.tag = name
            callback.writer.add_summary(summary, batch_no)
            callback.writer.flush()
    
    def get_discriminator(self, model, weights=None):
        
        if not self.disc_flag:
            self.define_discriminator(model.output_shape[1:])
        
        disc = Model(inputs=(model.input), outputs=(self.discriminator_model(model.output)))
        
        if weights is not None:
            disc.load_weights(weights, by_name=True)
        
        return disc
      
    def train_source_model(self, model, epochs=2000, batch_size=128, save_interval=1, start_epoch=0):

        (train_x, train_y), (test_x, test_y) = get_dataset('drive/My Drive/pro_data/dataset_shelly')
        
        datagen = ImageDataGenerator(data_format='channels_last', 
                                rescale=1./255, 
                                rotation_range=40, 
                                width_shift_range=0.2, 
                                height_shift_range=0.2)
       
        evalgen = ImageDataGenerator(data_format='channels_last', 
                                rescale=1./255)
        
        model.compile(loss='categorical_crossentropy', optimizer=self.src_optimizer, metrics=['accuracy'])
        
        if not os.path.isdir('data'):
            os.mkdir('data')
        
        # saver = keras.callbacks.ModelCheckpoint('drive/My Drive/pro_data/source_encoder_{epoch:02d}.hdf5', 
        #                                 monitor='val_loss', 
        #                                 verbose=1, 
        #                                 save_best_only=False, 
        #                                 save_weights_only=True, 
        #                                 mode='auto', 
        #                                 period=save_interval)

        # scheduler = keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.75, patience=10, verbose=0, mode='min')

        if not os.path.isdir('drive/My Drive/pro_data/tensorboard'):
            os.mkdir('drive/My Drive/pro_data/tensorboard')
    
        # visualizer = keras.callbacks.TensorBoard(log_dir=os.path.join('drive/My Drive/pro_data/tensorboard'), 
        #                                     histogram_freq=0, 
        #                                     write_graph=True, 
        #                                     write_images=False)
        
        # model.fit_generator(datagen.flow(train_x, train_y, batch_size=batch_size, shuffle=True),
        #                     steps_per_epoch=2000, 
        #                     epochs=epochs,
        #                     callbacks=[saver, scheduler, visualizer], 
        #                     validation_data=evalgen.flow(test_x, test_y, batch_size=batch_size), 
        #                     initial_epoch=start_epoch)

        model.fit_generator(datagen.flow(train_x, train_y, batch_size=batch_size, shuffle=True),
                            steps_per_epoch=2000, 
                            epochs=epochs, 
                            validation_data=evalgen.flow(test_x, test_y, batch_size=batch_size), 
                            initial_epoch=start_epoch)
        
    def train_target_discriminator(self, source_model=None, src_discriminator=None, tgt_discriminator=None, epochs=2000, batch_size=100, save_interval=1, start_epoch=0, num_batches=100):   
    
        (source_x, _), (_,_) = get_dataset('drive/My Drive/pro_data/dataset_shelly')
        
        src_datagen = ImageDataGenerator(data_format='channels_last', 
                                rescale=1./255, 
                                rotation_range=40, 
                                width_shift_range=0.2, 
                                height_shift_range=0.2)
        
        (target_x, _), (_,_) = get_dataset('drive/My Drive/pro_data/dataset_yerus')
        
        tgt_datagen = ImageDataGenerator(data_format='channels_last', 
                                rescale=1./255, 
                                rotation_range=40, 
                                width_shift_range=0.2, 
                                height_shift_range=0.2)
        
        self.define_source_encoder(source_model)
                
        for layer in self.source_encoder.layers:
            layer.trainable = False
        
        source_discriminator = self.get_discriminator(self.source_encoder, src_discriminator)
        target_discriminator = self.get_discriminator(self.target_encoder, tgt_discriminator)
        
        if src_discriminator is not None:
            source_discriminator.load_weights(src_discriminator)
        if tgt_discriminator is not None:
            target_discriminator.load_weights(tgt_discriminator)
        
        source_discriminator.compile(loss = "binary_crossentropy", optimizer=self.tgt_optimizer, metrics=['accuracy'])
        target_discriminator.compile(loss = "binary_crossentropy", optimizer=self.tgt_optimizer, metrics=['accuracy'])
        
        # callback1 = keras.callbacks.TensorBoard('data/tensorboard')
        # callback1.set_model(source_discriminator)
        # callback2 = keras.callbacks.TensorBoard('data/tensorboard')
        # callback2.set_model(target_discriminator)
        src_names = ['src_discriminator_loss', 'src_discriminator_acc']
        tgt_names = ['tgt_discriminator_loss', 'tgt_discriminator_acc']
        
        for iteration in range(start_epoch, epochs):
            
            avg_loss, avg_acc, index = [0, 0], [0, 0], 0

            for mnist,svhn in zip(src_datagen.flow(source_x, None, batch_size=batch_size), tgt_datagen.flow(target_x, None, batch_size=batch_size)):
                s_domain = np_utils.to_categorical(np.zeros(mnist.shape[0]), 2)
                t_domain =  np_utils.to_categorical(np.ones(svhn.shape[0]), 2)
                l1, acc1 = source_discriminator.train_on_batch(mnist,s_domain )
                l2, acc2 = target_discriminator.train_on_batch(svhn,t_domain)
                index+=1
                loss, acc = (l1+l2)/2, (acc1+acc2)/2
                print (iteration+1,': ', index,'/', num_batches, '; Loss: %.4f'%loss, ' (', '%.4f'%l1, '%.4f'%l2, '); Accuracy: ', acc, ' (', '%.4f'%acc1, '%.4f'%acc2, ')')
                avg_loss[0] += l1
                avg_acc[0] += acc1
                avg_loss[1] += l2
                avg_acc[1] += acc2
                if index%num_batches == 0:
                    break
            
            if iteration%self.discriminator_decay_rate==0:
                lr = K.get_value(source_discriminator.optimizer.lr)
                K.set_value(source_discriminator.optimizer.lr, lr*self.discriminator_decay_factor)
                lr = K.get_value(target_discriminator.optimizer.lr)
                K.set_value(target_discriminator.optimizer.lr, lr*self.discriminator_decay_factor)
                print ('Learning Rate Decayed to: ', K.get_value(target_discriminator.optimizer.lr))
            
            if iteration%save_interval==0:
                source_discriminator.save_weights('drive/My Drive/pro_data/discriminator_mnist_%02d.hdf5'%iteration)
                target_discriminator.save_weights('drive/My Drive/pro_data/discriminator_svhn_%02d.hdf5'%iteration)
                
            # self.tensorboard_log(callback1, src_names, [avg_loss[0]/mnist.shape[0], avg_acc[0]/mnist.shape[0]], iteration)
            # self.tensorboard_log(callback2, tgt_names, [avg_loss[1]/mnist.shape[0], avg_acc[1]/mnist.shape[0]], iteration)
    
    def eval_source_classifier(self, model, dataset='drive/My Drive/pro_data/dataset_shelly', batch_size=128, domain='Source'):
        
        (train_x,_), (test_x, test_y) = get_dataset(dataset)
        src_datagen = ImageDataGenerator(data_format='channels_last', 
                                        rescale=1./255)
                      
        model.compile(loss='categorical_crossentropy', optimizer=self.src_optimizer, metrics=['accuracy'])

        scores = model.evaluate_generator(src_datagen.flow(test_x[:10000], test_y[:10000]),10000)
        print('%s %s Classifier Test loss:%.5f'%(dataset.upper(), domain, scores[0]))
        print('%s %s Classifier Test accuracy:%.2f%%'%(dataset.upper(), domain, float(scores[1])*100))            
            
    def eval_target_classifier(self, source_model, target_discriminator, dataset='drive/My Drive/pro_data/dataset_shelly'):
        
        self.define_target_encoder()
        model = self.get_source_classifier(self.target_encoder, source_model)
        model.load_weights(target_discriminator, by_name=True)
        model.summary()
        self.eval_source_classifier(model, dataset=dataset, domain='Target')
         
if __name__ == '__main__':

    # ap = argparse.ArgumentParser()
    # ap.add_argument('-s', '--source_weights', required=False, help="Path to weights file to load source model for training classification/adaptation")
    # ap.add_argument('-e', '--start_epoch', type=int,default=1, required=False, help="Epoch to begin training source model from")
    # ap.add_argument('-n', '--discriminator_epochs', type=int, default=10000, help="Max number of steps to train discriminator")
    # ap.add_argument('-l', '--lr', type=float, default=0.0001, help="Initial Learning Rate")
    # ap.add_argument('-f', '--train_discriminator', action='store_true', help="Train discriminator model (if TRUE) vs Train source classifier")
    # ap.add_argument('-a', '--source_discriminator_weights', help="Path to weights file to load source discriminator")
    # ap.add_argument('-b', '--target_discriminator_weights', help="Path to weights file to load target discriminator")
    # ap.add_argument('-t', '--eval_source_classifier', default=None, help="Path to source classifier model to test/evaluate")
    # ap.add_argument('-d', '--eval_target_classifier', default=None, help="Path to target discriminator model to test/evaluate")
    # args = ap.parse_args()
    
    adda = ADDA(0.0001)
    adda.define_source_encoder()

    start_epoch=1
    train_discriminator=True
    discriminator_epochs=10000
    source_weights = None
    source_discriminator_weights=None
    target_discriminator_weights=None
    eval_source_classifier = None 
    #No path to source classifier model to test/evaluate

    if not train_discriminator:
        if eval_source_classifier is None:
            model = adda.get_source_classifier(adda.source_encoder, source_weights)
            adda.train_source_model(model, start_epoch-1) 
        else:
            model = adda.get_source_classifier(adda.source_encoder, None)
            adda.eval_source_classifier(model, 'mnist')
            adda.eval_source_classifier(model, 'svhn')
    adda.define_target_encoder(source_weights)
    
    if train_discriminator:
        adda.train_target_discriminator(epochs=discriminator_epochs, 
                                        source_model=source_weights, 
                                        src_discriminator=source_discriminator_weights, 
                                        tgt_discriminator=target_discriminator_weights,
                                        start_epoch=start_epoch-1)
    # if args.eval_target_classifier is not None:
    #     adda.eval_target_classifier(args.eval_source_classifier, args.eval_target_classifier)

[INFO] loading images...
['drive/My Drive/pro_data/dataset_shelly/no/20150910_095805.jpg', 'drive/My Drive/pro_data/dataset_shelly/no/20150913_184612.jpg', 'drive/My Drive/pro_data/dataset_shelly/no/20150913_184615.jpg', 'drive/My Drive/pro_data/dataset_shelly/no/20150913_184618.jpg', 'drive/My Drive/pro_data/dataset_shelly/no/20150913_184622.jpg', 'drive/My Drive/pro_data/dataset_shelly/no/20150913_184626.jpg', 'drive/My Drive/pro_data/dataset_shelly/no/20150913_184904.jpg', 'drive/My Drive/pro_data/dataset_shelly/no/20151011_152558.jpg', 'drive/My Drive/pro_data/dataset_shelly/no/20151011_152642.jpg', 'drive/My Drive/pro_data/dataset_shelly/no/20151018_212830.jpg', 'drive/My Drive/pro_data/dataset_shelly/no/20151024_113302.jpg', 'drive/My Drive/pro_data/dataset_shelly/no/20151111_195411.jpg', 'drive/My Drive/pro_data/dataset_shelly/no/20151203_144530.jpg', 'drive/My Drive/pro_data/dataset_shelly/no/20151203_144604.jpg', 'drive/My Drive/pro_data/dataset_shelly/no/20160422_221701.jpg',

/usr/local/lib/python3.6/dist-packages/keras/activations.py:235: UserWarning: Do not pass a layer instance (such as LeakyReLU) as the activation argument of another layer. Instead, advanced activation layers should be used just like any other layer in a model.
  identifier=identifier.__class__.__name__))


1 :  1 / 100 ; Loss: 3.2337  ( 3.2540 3.2135 ); Accuracy:  0.17000000178813934  ( 0.3400 0.0000 )


FailedPreconditionError: ignored

In [11]:
pip install keras_vggface

In [0]:

from keras.layers import Input, Dense, Reshape, Flatten, Dropout
from keras.layers import BatchNormalization, Activation
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model, clone_model
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam

import keras
from keras.utils import np_utils

import tensorflow.compat.v1 as tf
# from datasets import get_dataset

import matplotlib.pyplot as plt

import sys
import os
import numpy as np
import argparse
from keras_vggface.vggface import VGGFace

In [0]:
def define_source_encoder():

        inp = Input(shape= param["inp_dims"])
        network = eval('VGGFace')
        base = network(weights = 'vggface', include_top = False)
        param["source_encoder"] = Model(inp,base(inp))

def define_target_encoder():
       param["target_encoder"] = clone_model(param["source_encoder"])

def get_source_classifier(model):
        
    x = Flatten()(model.output)
    dense1 = Dense(400, name = 'class_dense1')(x)
    bn1 = BatchNormalization(name = 'class_bn1')(dense1)
    act1 = Activation('relu', name = 'class_act1')(bn1)
    drop2 = Dropout(param["drop_classifier"], name = 'class_drop1')(act1)

    dense2 = Dense(100, name = 'class_dense2')(drop2)
    bn2 = BatchNormalization(name = 'class_bn2')(dense2)
    act2 = Activation('relu', name = 'class_act2')(bn2)
    drop2 = Dropout(param["drop_classifier"], name = 'class_drop2')(act2)
    densel = Dense(param["number_of_classe"], name = 'class_dense_last')(drop2)
    bnl = BatchNormalization(name = 'class_bn_last')(densel)
    actl = Activation('softmax', name = 'class_act_last')(bnl)
        
    source_classifier_model = Model(inputs=(model.input), outputs=(actl))
        
    return source_classifier_model

def define_discriminator(shape):
        
    inp = Input(shape=shape)
    x = Flatten()(inp)
    dense1 = Dense(400, name = 'dis_dense1')(x)
    bn1 = BatchNormalization(name='dis_bn1')(dense1)
    act1 = Activation('relu', name = 'dis_act1')(bn1)
    drop1 = Dropout(param["drop_discriminator"], name = 'dis_drop1')(act1)

    dense2 = Dense(100, name = 'dis_dense2')(drop1)
    bn2 = BatchNormalization(name='dis_bn2')(dense2)
    act2 = Activation('relu', name = 'dis_act2')(bn2)
    drop2 = Dropout(param["drop_discriminator"], name = 'dis_drop2')(act2)

    densel = Dense(1, name = 'dis_dense_last')(drop2)
    bnl = BatchNormalization(name = 'dis_bn_last')(densel)
    actl = Activation('sigmoid', name = 'dis_act_last')(bnl)
    param["discriminator_model"] = Model(inputs=(inp), outputs=(actl), name='discriminator')

def get_discriminator(model):    
    disc = Model(inputs=(model.input), outputs=(param["discriminator_model"](model.output)))
    return disc

  

In [0]:
def train_source_model(model, epochs=2000, batch_size=128, save_interval=1, start_epoch=0):

        (train_x, train_y), (test_x, test_y) = get_dataset('drive/My Drive/pro_data/dataset_shelly')
        
        datagen = ImageDataGenerator(data_format='channels_last', 
                                rescale=1./255, 
                                rotation_range=40, 
                                width_shift_range=0.2, 
                                height_shift_range=0.2)
       
        evalgen = ImageDataGenerator(data_format='channels_last', 
                                rescale=1./255)
        
        model.compile(optimizer = opt_combined(param), loss = ['categorical_crossentropy', 'binary_crossentropy'] , loss_weights =  [param["class_loss_weight"],  param["dis_loss_weight"]], metrics = ['accuracy'])

        model.fit_generator(datagen.flow(train_x, train_y, batch_size=batch_size, shuffle=True),
                            steps_per_epoch=2000, 
                            epochs=epochs, 
                            validation_data=evalgen.flow(test_x, test_y, batch_size=batch_size), 
                            initial_epoch=start_epoch)

In [0]:
def train_target_discriminator( epochs=2000, batch_size=100, save_interval=1, start_epoch=0, num_batches=100):   
    
        (source_x, _), (_,_) = get_dataset('drive/My Drive/pro_data/dataset_shelly')
        
        src_datagen = ImageDataGenerator(data_format='channels_last', 
                                rescale=1./255, 
                                rotation_range=40, 
                                width_shift_range=0.2, 
                                height_shift_range=0.2)
        
        (target_x, _), (_,_) = get_dataset('drive/My Drive/pro_data/dataset_yerus')
        
        tgt_datagen = ImageDataGenerator(data_format='channels_last', 
                                rescale=1./255, 
                                rotation_range=40, 
                                width_shift_range=0.2, 
                                height_shift_range=0.2)
        
        define_source_encoder()
        define_target_encoder()
        define_discriminator(param["source_encoder"].output_shape[1:])
                
        for layer in param["source_encoder"].layers:
            layer.trainable = False
        
        param["source_dicriminator"] = get_discriminator(param["source_encoder"])
        param["target_dicriminator"]= get_discriminator(param["target_encoder"])
        
        
        param["source_dicriminator"].compile(optimizer = opt_discriminator(param), loss = 'binary_crossentropy', metrics = ['accuracy'])
        param["target_dicriminator"].compile(optimizer = opt_discriminator(param), loss = 'binary_crossentropy', metrics = ['accuracy'])
        
        src_names = ['src_discriminator_loss', 'src_discriminator_acc']
        tgt_names = ['tgt_discriminator_loss', 'tgt_discriminator_acc']
        
        for iteration in range(start_epoch, epochs):
            
            avg_loss, avg_acc, index = [0, 0], [0, 0], 0

            for source,target in zip(src_datagen.flow(source_x, None, batch_size=batch_size), tgt_datagen.flow(target_x, None, batch_size=batch_size)):
                # s_domain = np_utils.to_categorical(np.zeros(source.shape[0]), 1)
                # t_domain =  np_utils.to_categorical(np.ones(target.shape[0]), 1)
                s_domain = np.zeros(source.shape[0])
                t_domain =  np.ones(target.shape[0])
                l1, acc1 = param["source_dicriminator"].train_on_batch(source,s_domain )
                l2, acc2 = param["target_dicriminator"].train_on_batch(target,t_domain)
                index+=1
                loss, acc = (l1+l2)/2, (acc1+acc2)/2
                print (iteration+1,': ', index,'/', num_batches, '; Loss: %.4f'%loss, ' (', '%.4f'%l1, '%.4f'%l2, '); Accuracy: ', acc, ' (', '%.4f'%acc1, '%.4f'%acc2, ')')
                avg_loss[0] += l1
                avg_acc[0] += acc1
                avg_loss[1] += l2
                avg_acc[1] += acc2
                if index%num_batches == 0:
                    break
            
            
            # if iteration%save_interval==0:
            #     param["source_dicriminator"].save_weights('drive/My Drive/pro_data/discriminator_s_%02d.hdf5'%iteration)
            #     param["target_dicriminator"].save_weights('drive/My Drive/pro_data/discriminator_t_%02d.hdf5'%iteration)
                

In [0]:
def eval_source_classifier( model, dataset, batch_size=128, domain='Source'):
        
        (train_x,_), (test_x, test_y) = get_dataset(dataset)
        src_datagen = ImageDataGenerator(data_format='channels_last', 
                                        rescale=1./255)
                      
        model.compile(optimizer = opt_combined(param), loss = ['categorical_crossentropy', 'binary_crossentropy'] , loss_weights =  [param["class_loss_weight"],  param["dis_loss_weight"]], metrics = ['accuracy'])

        scores = model.evaluate_generator(src_datagen.flow(test_x[:10000], test_y[:10000]),10000)
        print('%s %s Classifier Test loss:%.5f'%(dataset.upper(), domain, scores[0]))
        print('%s %s Classifier Test accuracy:%.2f%%'%(dataset.upper(), domain, float(scores[1])*100))            
            
def eval_target_classifier( target_discriminator, dataset):
        
        define_target_encoder()
        model = get_source_classifier(paran["target_encoder"])
        model.load_weights(target_discriminator, by_name=True)
        model.summary()
        eval_source_classifier(model, dataset=dataset, domain='Target')

In [0]:
import argparse
if __name__ == "__main__":
    # Read parameter values from the console
    parser = argparse.ArgumentParser(description = 'Domain Adaptation')
    parser.add_argument('--number_of_gpus', type = int, nargs = '?', default = '1', help = "Number of gpus to run")
    parser.add_argument('--network_name', type = str, default = 'ResNet50', help = "Name of the feature extractor network")
    parser.add_argument('--dataset_name', type = str, default = 'Office', help = "Name of the source dataset")
    parser.add_argument('--dropout_classifier', type = float, default = 0.25, help = "Dropout ratio for classifier")
    parser.add_argument('--dropout_discriminator', type = float, default = 0.25, help = "Dropout ratio for discriminator")    
    parser.add_argument('--source_path', type = str, default = 'amazon_10_list.txt', help = "Path to source dataset")
    parser.add_argument('--target_path', type = str, default = 'webcam_10_list.txt', help = "Path to target dataset")
    parser.add_argument('--lr_classifier', type = float, default = 0.0001, help = "Learning rate for classifier model")
    parser.add_argument('--b1_classifier', type = float, default = 0.9, help = "Exponential decay rate of first moment \
                                                                                             for classifier model optimizer")
    parser.add_argument('--b2_classifier', type = float, default = 0.999, help = "Exponential decay rate of second moment \
                                                                                            for classifier model optimizer")
    parser.add_argument('--lr_discriminator', type = float, default = 0.00001, help = "Learning rate for discriminator model")
    parser.add_argument('--b1_discriminator', type = float, default = 0.9, help = "Exponential decay rate of first moment \
                                                                                             for discriminator model optimizer")
    parser.add_argument('--b2_discriminator', type = float, default = 0.999, help = "Exponential decay rate of second moment \
                                                                                            for discriminator model optimizer")
    parser.add_argument('--lr_combined', type = float, default = 0.00001, help = "Learning rate for combined model")
    parser.add_argument('--b1_combined', type = float, default = 0.9, help = "Exponential decay rate of first moment \
                                                                                             for combined model optimizer")
    parser.add_argument('--b2_combined', type = float, default = 0.999, help = "Exponential decay rate of second moment \
                                                                                            for combined model optimizer")
    parser.add_argument('--classifier_loss_weight', type = float, default = 1, help = "Classifier loss weight")
    parser.add_argument('--discriminator_loss_weight', type = float, default = 4, help = "Discriminator loss weight")
    parser.add_argument('--batch_size', type = int, default = 32, help = "Batch size for training")
    parser.add_argument('--test_interval', type = int, default = 3, help = "Gap between two successive test phases")
    parser.add_argument('--num_iterations', type = int, default = 120, help = "Number of iterations")
    parser.add_argument('--snapshot_interval', type = int, default = 500, help = "Minimum gap between saving outputs")
    parser.add_argument('--output_dir', type = str, default = 'Models', help = "Directory for saving outputs")
    # args = parser.parse_args()

    # Set GPU device
    # os.environ["CUDA_VISIBLE_DEVICES"] = str(list(np.arange(args.number_of_gpus))).strip('[]')

    # Initialize parameters
    param = {}
    param["number_of_gpus"] = 1
    # param["network_name"] = 'ResNet50'
    param["network_name"] = 'VGGFace'
    param["inp_dims"] = [224, 224, 3]
    param["num_iterations"] = 10000
    # param["num_iterations"] = 500
    param["lr_classifier"] = 0.0001
    param["b1_classifier"] = 0.9
    param["b2_classifier"] = 0.999    
    param["lr_discriminator"] = 0.00001
    param["b1_discriminator"] =  0.9
    param["b2_discriminator"] = 0.999
    param["lr_combined"] = 0.00001
    param["b1_combined"] =  0.9
    param["b2_combined"] =  0.999       
    # param["batch_size"] = int(32)
    param["batch_size"] = int(32/2)
    param["class_loss_weight"] = 1
    param["dis_loss_weight"] = 4    
    param["drop_classifier"] = 0.25
    param["drop_discriminator"] = 0.25
    param["test_interval"] = 100
    # param["source_path"] = 'drive/My Drive/final_proj_dataset/data_file_shelly.txt'
    # param["target_path"] = 'drive/My Drive/final_proj_dataset/data_file_yerus.txt' 
    param["source_path"] = 'drive/My Drive/pro_data/source'
    param["target_path"] = 'drive/My Drive/pro_data/target' 
    param["snapshot_interval"] = 500
    # param["snapshot_interval"] = 5
    param["output_path"] = 'drive/My Drive/pro_data/result_vgg'
    param["number_of_classe"] = 0

    param["source_encoder"] = None
    param["target_encoder"] = None
    param["target_dicriminator"] = None
    param["source_dicriminator"] = None
    param["discriminator_model"] = None

    train_target_discriminator()
    eval_target_classifier(param["target_dicriminator"],'drive/My Drive/pro_data/dataset_shelly')
    # model = get_source_classifier(param["source_encoder"])
    # eval_source_classifier(model, 'drive/My Drive/pro_data/dataset_shelly')
    # eval_source_classifier(model, 'drive/My Drive/pro_data/dataset_yerus')
    
    
    
    

   

[INFO] loading images...
['drive/My Drive/pro_data/dataset_shelly/no/20150910_095805.jpg', 'drive/My Drive/pro_data/dataset_shelly/no/20150913_184612.jpg', 'drive/My Drive/pro_data/dataset_shelly/no/20150913_184615.jpg', 'drive/My Drive/pro_data/dataset_shelly/no/20150913_184618.jpg', 'drive/My Drive/pro_data/dataset_shelly/no/20150913_184622.jpg', 'drive/My Drive/pro_data/dataset_shelly/no/20150913_184626.jpg', 'drive/My Drive/pro_data/dataset_shelly/no/20150913_184904.jpg', 'drive/My Drive/pro_data/dataset_shelly/no/20151011_152558.jpg', 'drive/My Drive/pro_data/dataset_shelly/no/20151011_152642.jpg', 'drive/My Drive/pro_data/dataset_shelly/no/20151018_212830.jpg', 'drive/My Drive/pro_data/dataset_shelly/no/20151024_113302.jpg', 'drive/My Drive/pro_data/dataset_shelly/no/20151111_195411.jpg', 'drive/My Drive/pro_data/dataset_shelly/no/20151203_144530.jpg', 'drive/My Drive/pro_data/dataset_shelly/no/20151203_144604.jpg', 'drive/My Drive/pro_data/dataset_shelly/no/20160422_221701.jpg',